<p style="background:black">
<code style="background:black;color:white">~/Users/YOUR_USERNAME/> pip3 install selenium<br> -->Voile
</code>
</p>

<div id="id"></div> 

[CONFTEST](#conftest) | [LOCATORS](#locators) | [PAGE OBJECTS](#page_obj) | [TESTS](#tests) | [RUNNERS](#runs)
     

<h3>
    <p id="conftest" style="font-family:Comic Sans MS;color:black;"> CONFEST </p>     
</h3>  

[-->](#id)

In [1]:
%%writefile conftest.py

import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def pytest_addoption(parser):
    parser.addoption('--browser_name', action='store', default="chrome",
                     help="Choose browser: chrome or firefox")
    parser.addoption('--language', action='store', default="en",
                     help="Choose language")
    
@pytest.fixture(scope="function")
def language(request):
    language = request.config.getoption("language")
    return language 

@pytest.fixture(scope="function")
def browser(request,language):
    
    browser_name = request.config.getoption("browser_name")
    browser = None
    
    if browser_name == "chrome":
        
        print("\nstart chrome browser for test..")
        options = Options()
        options.add_experimental_option('prefs', {'intl.accept_languages': language})
        browser = webdriver.Chrome(options=options)
        
    elif browser_name == "firefox":
        
        print("\nstart firefox browser for test..")
        fp = webdriver.FirefoxProfile()
        fp.set_preference("intl.accept_languages", language)
        browser = webdriver.Firefox(firefox_profile=fp)

    else:
        
        raise pytest.UsageError("--browser_name should be chrome or firefox")
        
    yield browser
    print("\nquit browser..")
    browser.quit()

    

Overwriting conftest.py


<h3>
    <div id="locators"  style="font-family:Comic Sans MS;color:black" > LOCATORS </div>
</h3>

[-->](#id)

In [2]:
%%writefile pages/locators.py

from selenium.webdriver.common.by import By

class MainPageLocators():
    LOGIN_LINK = (By.CSS_SELECTOR, "#login_link")
    
class LoginPageLocators():
    LOGIN_FORM = (By.CSS_SELECTOR, "#login_form")
    REGISTER_FORM = (By.CSS_SELECTOR, "#register_form")

class ProductPageLocators():
    BASKET_BUTTON = (By.CSS_SELECTOR,"button.btn-add-to-basket")
    PRODUCT_NAME  = (By.CSS_SELECTOR,".product_main > h1")
    BUSKET_PRODUCT_NAME  = (By.CSS_SELECTOR,".alertinner > strong")
    PRODUCT_PRICE = (By.CSS_SELECTOR,".product_main > .price_color")
    BUSKET_PRODUCT_PRICE  = (By.CSS_SELECTOR,".alertinner > p > strong")
    SUCCESS_MESSAGE = (By.CSS_SELECTOR,"#messages > div:nth-child(1)")

Overwriting pages/locators.py


<h3>
    <p id="page_obj" style="font-family:Comic Sans MS;color:black"> PAGE OBJECTS </p> 
</h3>

[-->](#id)

[BasePage](#BasePage) | [LoginPage](#LoginPage) | [MainPage](#MainPage) | [ProductPage](#ProductPage) |

> <h4>
    <p id="BasePage" style="font-family:Comic Sans MS;color:black" > BasePage </p>
</h4>

In [3]:
%%writefile pages/base_page.py

from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math

class BasePage():
    
    def __init__(self, browser, url, timeout=10):
        
        self.browser = browser
        self.url = url
        self.browser.implicitly_wait(timeout)
        
        
    def open(self):
        
        self.browser.get(self.url)
        
        
    def is_element_present(self, how, what):
        
        try:
            self.browser.find_element(how, what)
        except NoSuchElementException:
            return False
        return True 
    
    
    def is_not_element_present(self, how, what, timeout=4):
            
        try:
            WebDriverWait(self.browser, timeout).until(EC.presence_of_element_located((how, what)))
        except TimeoutException:
            return True
        return False  
    
    
    def solve_quiz_and_get_code(self):
        
        alert = self.browser.switch_to.alert
        x = int(alert.text.split(" ")[2])
        answer = str(math.log(abs((12 * math.sin(float(x))))))
        alert.send_keys(answer)
        alert.accept()
        
        try:
            alert = self.browser.switch_to.alert
            alert_text = alert.text
            print(f"Your code: {alert_text}")
            alert.accept()
            
        except NoAlertPresentException:
            print("No second alert presented")
            
            
    def is_disappeared(self, how, what, timeout=4):
        
        try:
            WebDriverWait(self.browser, timeout, 1, TimeoutException).\
            until_not(EC.presence_of_element_located((how, what)))
        except TimeoutException:
            return False
        return True  

Overwriting pages/base_page.py


> <h4>
    <p id="LoginPage" style="font-family:Comic Sans MS;color:black" > LoginPage </p>
</h4>

In [4]:
%%writefile pages/login_page.py

from .base_page import BasePage
from .locators import LoginPageLocators

class LoginPage(BasePage):
    
    def should_be_login_page(self):
        self.should_be_login_url()
        self.should_be_login_form()
        self.should_be_register_form()

    def should_be_login_url(self):
        
        assert "login" in self.browser.current_url, 'Incorrected URL (not inclusive substring "login").'

    def should_be_login_form(self):
        
        assert self.browser.find_element(*LoginPageLocators.LOGIN_FORM) ,'Up-s-s...Login form not found!'

    def should_be_register_form(self):
        
        assert self.browser.find_element(*LoginPageLocators.REGISTER_FORM),'Up-s-s...Register form not found!'

Overwriting pages/login_page.py


> <h4>
    <p id="MainPage" style="font-family:Comic Sans MS;color:black"> MainPage </p>
</h4>

In [5]:
%%writefile pages/main_page.py

from selenium.webdriver.common.by import By
from .base_page import BasePage
from .locators import MainPageLocators
from .login_page import LoginPage

class MainPage(BasePage): 
    
    def go_to_login_page(self):
        
        link = self.browser.find_element(*MainPageLocators.LOGIN_LINK)
        link.click()        
        return LoginPage(browser=self.browser, url=self.browser.current_url) 
                     
    def should_be_login_link(self):
        assert self.is_element_present(*MainPageLocators.LOGIN_LINK), "Login link is not presented"
        

Overwriting pages/main_page.py


> <h4>
    <p id="ProductPage" style="font-family:Comic Sans MS;color:black"> ProductPage </p>
</h4>

In [6]:
%%writefile pages/product_page.py

from selenium.webdriver.common.by import By
from .base_page import BasePage
from .locators import ProductPageLocators

class ProductPage(BasePage):
    
    def add_product_to_basket(self):
        self.should_be_button()
        self.solve_quiz_and_get_code()
        self.should_be_equal_name()
        self.should_be_equal_price()
    
    def should_be_button(self): 
        
        button = self.browser.find_element(*ProductPageLocators.BASKET_BUTTON)
        button.click()
        print('Your code: Congratulations! Buttton presented!--> ')
    
    def should_be_equal_name(self):
        
        product_name  = self.browser.find_element(*ProductPageLocators.PRODUCT_NAME)
        basket_name  = self.browser.find_element(*ProductPageLocators.BUSKET_PRODUCT_NAME)
        assert basket_name.text == product_name.text, 'not equal names!!'
        print('Your code: Congratulations! Couple name is equal! --> ')
        
    def should_be_equal_price(self):
        
        product_price = self.browser.find_element(*ProductPageLocators.PRODUCT_PRICE)
        basket_price  = self.browser.find_element(*ProductPageLocators.BUSKET_PRODUCT_PRICE)
        assert basket_price.text == product_price.text, 'not equal prices!!'
        print('Your code: Congratulations! Couple price is equal! --> ')
        
    def should_not_be_success_message(self):
        assert self.is_not_element_present(*ProductPageLocators.SUCCESS_MESSAGE), \
           "Success message is presented, but should not be" 
        print('Your code: Congratulations! Success message not presented! --> ')
        
    def should_be_success_message(self):
        assert self.is_element_present(*ProductPageLocators.SUCCESS_MESSAGE), \
           "Success message not presented, but should be!"  
        print('Your code: Congratulations! Success message presented! --> ')
        
    def success_message_should_disappear(self):
        assert True
        print('Your code: Congratulations! Success message disappeared! --> ')

Overwriting pages/product_page.py


<h3>
    <p id="tests" style="font-family:Comic Sans MS;color:black"> TESTS </p>
</h3>

[-->](#id)

[test_main_page](#test_main_page) | [test_product_page](#test_product_page)

> <h4>
    <p id="test_main_page" style="font-family:Comic Sans MS;color:black"> test_main_page </p>
</h4>

In [7]:
%%writefile test_main_page.py

from pages.main_page import MainPage

link = "http://selenium1py.pythonanywhere.com/"

class Test_main_page():
    
    def test_guest_should_see_login_link(self, browser):

        page = MainPage(browser, link)
        page.open()
        page.should_be_login_link()  


    def test_guest_can_go_to_login_page(self,browser):
        
        page = MainPage(browser, link)
        page.open()
        login_page = page.go_to_login_page()
        login_page.should_be_login_page()



Overwriting test_main_page.py


> <h4>
    <p id="test_product_page" style="font-family:Comic Sans MS;color:black"> test_product_page </p>
</h4>

In [8]:
%%writefile test_product_page.py

# import pytest
import time

from pages.product_page import ProductPage

link = " http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=newYear2019"

# class Test_product_page():
    
# @pytest.mark.parametrize(
#         'link', ["http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer0",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer1",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer2",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer3",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer4",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer5",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer6",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer7",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer8",
#                  "http://selenium1py.pythonanywhere.com/catalogue/coders-at-work_207/?promo=offer9"])
    
def  test_guest_can_add_product_to_basket(browser):

    try:
        page = ProductPage(browser,link)
        page.open()
        page.should_not_be_success_message()
        page.add_product_to_basket()
        page.should_be_success_message()
        page.success_message_should_disappear()
        
    finally:    
        time.sleep(1)     

Overwriting test_product_page.py


<h3>
    <p id="runs" style="font-family:Comic Sans MS;color:black" > RUNNERS </p>
</h3>

<h4><p id="runs" style="font-family:Comic Sans MS;color:black" > test_main_page </p></h4>

In [9]:
!pytest -sv --tb=line --language=en test_main_page.py

============================= test session starts ==============================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-0.13.1 -- /Users/silinanata/environments/selenium_env/bin/python3
cachedir: .pytest_cache
rootdir: /Users/silinanata/selenium_project/first-selenium-project
plugins: rerunfailures-10.2
collected 2 items                                                              

test_main_page.py::Test_main_page::test_guest_should_see_login_link 
start chrome browser for test..
PASSED
quit browser..

test_main_page.py::Test_main_page::test_guest_can_go_to_login_page 
start chrome browser for test..
PASSED
quit browser..


============================== 2 passed in 9.38s ===============================


<h4><p id="runs" style="font-family:Comic Sans MS;color:black" > test_product_page </p></h4>

In [10]:
!pytest -sv --tb=line test_product_page.py

============================= test session starts ==============================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-0.13.1 -- /Users/silinanata/environments/selenium_env/bin/python3
cachedir: .pytest_cache
rootdir: /Users/silinanata/selenium_project/first-selenium-project
plugins: rerunfailures-10.2
collected 1 item                                                               

test_product_page.py::test_guest_can_add_product_to_basket 
start chrome browser for test..
Your code: Congratulations! Success message not presented! --> 
Your code: Congratulations! Buttton presented!--> 
Your code: Поздравляем, вы справились! Вставьте это число в поле ответа на Stepik: 27.26633731799304
Your code: Congratulations! Couple name is equal! --> 
Your code: Congratulations! Couple price is equal! --> 
Your code: Congratulations! Success message presented! --> 
Your code: Congratulations! Success message disappeared! --> 
PASSED
quit browser..


======================